# Open Loop: Basics

An example of scanning over LED brightnesses and shutter delays, to see impact on the photodiode

In [1]:
import numpy as np
import epics
from epics import PV
from time import sleep

## Setup

### Set all six LEDs to flash in unison

Create a six-element list of LED-duration process variables

In [2]:
LEDdurs = [None]*6
for i in range(6):
    LEDdurs[i] = PV('LEDS:CH' + str(i) + ':dur.VAL')

Set all LEDs to flash for the same duration: five hundred milliseconds.

In [3]:
for LEDdur in LEDdurs:
    LEDdur.put(500e3)

### Initialize all six LEDs at their maximum brightness

Create a six-element list of LED-brightness process variables

In [4]:
LEDbrigs = [None]*6
for i in range(6):
    LEDbrigs[i] = PV('LEDS:CH' + str(i) + ':brig.VAL')

Set all LEDs to the same brightness level: 255 (out of 255)

In [5]:
for LEDbrig in LEDbrigs:
    LEDbrig.put(255)

### Ensure the shutter is open throughout LED flash

Enable the shutter, if it isn't already enabled

In [6]:
epics.caput("SHUTTER:enable", 1)

1

Specify that the shutter should wait **a full 1.5 seconds** to start closing after opening

In [7]:
epics.caput("SHUTTER:dur", 1.5e6)

1

Reduce the overall repetition rate of the system to once every three seconds

In [8]:
epics.caput("PULSEGEN:reprate", 1/3.0)

1

Increase the delay time on the LED trigger signal (Channel 0 from the Pulse Generator) to be 400 milliseconds (it started at 2 milliseconds)

In [9]:
epics.caput("PULSEGEN:CH0:delay", 400.0e3) # Set LED trigger delay to 400 ms

1

Set the delay time on the Shutter trigger signal (Channel 1 from the Pulse Generator) to be 0 milliseconds, which is 400 milliseconds earlier than the LED trigger.

In [10]:
epics.caput("PULSEGEN:CH1:delay", 0.0e3) # Set Shutter trigger delay to 0 ms

1

### Set up the photodetector trigger delay and exposure time

We want the photodetector to start exposure before the LEDs begin flashing, and end exposure afterwards.

We will set the delay time on the photodetector trigger signal (Channel 2 from the Pulse Generator) to be 300 ms, which is 100 ms prior to the LEDs flashing.

In [11]:
epics.caput("PULSEGEN:CH2:delay", 300.0e3) # Set Photodetector trigger delay to 300 ms

1

Now, we will set the exposure length on the photodiode to a full second, to make sure we capture the entire LED flash duration.

In [12]:
epics.caput("PHOTO:dur", 1000.0e3) # Set photodetector exposure length to one second

1

### Read the photodetector value

In [13]:
epics.caget("PHOTO:data")

'DATA: 0.00640329, TRIGCNT: 91'

### Explore the impact of brightness values on photodetector signal

Demonstration of looping over brightness values

In [14]:
for brig in np.linspace(0, 255, 5).astype('int'):
    print("Brightness: ", brig)
    for LEDbrig in LEDbrigs:
        LEDbrig.put(brig)
    sleep(8)
    print(epics.caget("PHOTO:data"))

Brightness:  0
DATA: 0.00264000, TRIGCNT: 93
Brightness:  63
DATA: 0.00397553, TRIGCNT: 96
Brightness:  127
DATA: 0.00354588, TRIGCNT: 99
Brightness:  191
DATA: 0.00680188, TRIGCNT: 101
Brightness:  255
DATA: 0.00731953, TRIGCNT: 104


### Explore the impact of shutter delay time on photodetector signal

Looping over shutter delay values

In [16]:
for delay in np.linspace(0, 1000.0e3, 5).astype('int'):
    print("Shutter delay: ", delay*1e-3, " milliseconds.")
    epics.caput("PULSEGEN:CH1:delay", delay) # Set Shutter trigger delay to 400 ms 
    sleep(8)
    print(epics.caget("PHOTO:data"))

Shutter delay:  0.0  milliseconds.
DATA: 0.02285670, TRIGCNT: 114
Shutter delay:  250.0  milliseconds.
DATA: 0.00972400, TRIGCNT: 117
Shutter delay:  500.0  milliseconds.
DATA: 0.00691576, TRIGCNT: 119
Shutter delay:  750.0  milliseconds.
DATA: 0.00329741, TRIGCNT: 122
Shutter delay:  1000.0  milliseconds.
DATA: 0.00080753, TRIGCNT: 125
